Cargo las librerías:

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import sklearn as sk

Cargo los datos:

In [2]:
X_train = pd.read_csv('X_train.csv')
del X_train['Unnamed: 0']

X_test = pd.read_csv('X_test.csv')
del X_test['Unnamed: 0']

Defino una función que impute las variables continuas empleando un modelo de Random Forest.

En principio, emplear un modelo de KNN podría parecer interesante, pero me decanto por un modelo de Random Forest debido a que el modelo KNN se ve muy afectado por la conocida cómo "curse of dimensionality", mientras que el modelo de Random Forest no se ve afectado prácticamente por este problema.

Aunque a primera vista se podría pensar que la llamada "curse of dimensionality" no debería ser un problema, teniendo 14763 observaciones, tengo 34 variables.

Una solución sería estandarizar estas variables, aplicar un análisis de componentes principales, y después aplicar KNN, eligiendo el número de vecinos y de componentes mediante, por ejemplo, grid search (con cross-validation).

Otra alternativa, por la que yo me he decantado, es realizar la imputación empleando un modelo que no se vea afectado (o que apenas se vea afectado) por dicha "curse of dimensionality", como es un Random Forest, que demostraré a continuación que es una excelente elección:

In [3]:
def Imputar_RFR(objetivo_train,datos_train,datos_imputar):
    
    # Defino valores para los NAN y los infinitos,
    # asegurándome de que estos valores no coinciden con otros.
    # ESTO FUNCIONA SÓLO POR QUE ESTE MODELO ES UN RANDOM FOREST,
    # si el modelo fuese, por ejemplo, una regresión, realizar esta
    # transformación sería un gravísimo error.
    
    Min_t = np.min(np.min(datos_train))
    Min_i = np.min(np.min(datos_imputar))
    Min = np.min([Min_t,Min_i])
    
    Max_t = np.max(np.max(datos_train))
    Max_i = np.max(np.max(datos_imputar))
    Max = np.max([Max_t,Max_i])
    
    V1 = Min-(Max-Min)
    V2 = Min-(3/2)*(Max-Min)
    V3 = Min-(2)*(Max-Min)
    
    datos_train = np.nan_to_num(datos_train,nan=V2, posinf=V1, neginf=V3)
    datos_imputar = np.nan_to_num(datos_imputar,nan=V2, posinf=V1, neginf=V3)
    
    # Entreno un random forest con 100 árboles, usando 6 variables en cada árbol
    # y empleando cost-complexity pruning.
    
    model = RandomForestRegressor(n_estimators=100, n_jobs=-1, bootstrap = True,
                                  max_features=6, ccp_alpha=0.00025)
    
    model.fit(datos_train, objetivo_train)
    
    # Devuelvo las predicciones:
    
    return model.predict(datos_imputar)

Para comprobar que funciona de forma eficaz, voy a probar esta función en la siguiente variable:

In [4]:
X_train['Review Scores Rating'].head()

0    99.0
1     NaN
2     NaN
3    98.0
4     NaN
Name: Review Scores Rating, dtype: float64

Guardo las columnas que emplearé para la imputación. Emplearé todas las columnas, excepto aquellas que provienen de las variables Amenities y Features, ya que son bastante distintas al resto, la propia columna a imputar, y por supuesto no empleo la variable objetivo:

Lista de columnas sin la variable objetivo, Amenities y Features:

In [5]:
X_train.columns[0:34]

Index(['Host Response Time', 'Host Response Rate', 'Host Listings Count',
       'Host Total Listings Count', 'Host Verifications', 'Property Type',
       'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms', 'Beds',
       'Bed Type', 'Guests Included', 'Extra People', 'Minimum Nights',
       'Maximum Nights', 'Calendar Updated', 'Availability 30',
       'Availability 60', 'Availability 90', 'Availability 365',
       'Number of Reviews', 'Review Scores Rating', 'Review Scores Accuracy',
       'Review Scores Cleanliness', 'Review Scores Checkin',
       'Review Scores Communication', 'Review Scores Location',
       'Review Scores Value', 'Cancellation Policy',
       'Calculated host listings count', 'Reviews per Month', 'Square Feet',
       'Zona'],
      dtype='object')

Elimino la variable a imputar del listado de columnas:

In [6]:
# lista2 es una lista con todas las variables menos 'Review Scores Rating':

lista2 = list(X_train.columns[0:34])
lista2.remove('Review Scores Rating')

Realizo la imputación:

In [7]:
Predicciones = Imputar_RFR(
    X_train['Review Scores Rating'][X_train['Review Scores Rating'].isna()==False],
    X_train[lista2][X_train['Review Scores Rating'].isna()==False],
    X_train[lista2][X_train['Review Scores Rating'].isna()==False])
Predicciones

array([98.09175402, 97.77708759, 83.64716667, ..., 81.13442857,
       92.16294156, 99.68314774])

El error cuadrático medio entre los valores reales y los imputados (he "imputado" los valores que no faltan, para poder medir la eficacia del método):

In [8]:
sk.metrics.mean_squared_error(X_train['Review Scores Rating'][X_train['Review Scores Rating'].isna()==False],
                                  Predicciones)

3.067100133465432

Compruebo cuál hubiese sido este mismo error cuadrático medio, de haber imputado con la media:

In [9]:
# Genero un vector de predicciones que es una repetición de la media:

Predecir_media = np.ones(sum(X_train['Review Scores Rating'].isna()==False))*np.mean(X_train['Review Scores Rating'][X_train['Review Scores Rating'].isna()==False])

sk.metrics.mean_squared_error(X_train['Review Scores Rating'][X_train['Review Scores Rating'].isna()==False],
                                  Predecir_media)

80.11289285839447

Emplear un modelo de Random Forest demuestra ser una opción muy superior para realizar las imputaciones.

El siguiente bucle emplea esta función para imputar todos los valores del dataset de test, empleando el dataset de train para entrenar los modelos de Random Forest que imputarán los datos:

In [10]:
# Para cada columna:

for variable in list(X_train.columns):
    
    # Si, y sólo si la columna tiene valores faltantes:
    
    if sum(X_test[variable].isna()) != 0:
        print(variable)
        
        # Predigo los valores faltantes empleando un random forest:
        
        lista2 = list(X_train.columns)
        lista2.remove(variable)
        
        Predicciones = Imputar_RFR(
        X_train[variable][X_train[variable].isna()==False],
        X_train[lista2][X_train[variable].isna()==False],
        X_test[lista2][X_test[variable].isna()])
    
        # Sustituyo los valores faltantes por las predicciones:
        
        X_test.loc[X_test[variable].isna(),variable] = Predicciones

Host Response Rate
Host Listings Count
Host Total Listings Count
Bathrooms
Bedrooms
Beds
Review Scores Rating
Review Scores Accuracy
Review Scores Cleanliness
Review Scores Checkin
Review Scores Communication
Review Scores Location
Review Scores Value
Reviews per Month


Compruebo que he eliminado los valores faltantes de las primeras 34 columnas (las siguientes columnas pertenecen a Amenities y Features y no tienen valores faltantes):

In [11]:
X_test.iloc[:,0:34].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1477 entries, 0 to 1476
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Host Response Time              1477 non-null   int64  
 1   Host Response Rate              1477 non-null   float64
 2   Host Listings Count             1477 non-null   float64
 3   Host Total Listings Count       1477 non-null   float64
 4   Host Verifications              1477 non-null   int64  
 5   Property Type                   1477 non-null   float64
 6   Room Type                       1477 non-null   int64  
 7   Accommodates                    1477 non-null   int64  
 8   Bathrooms                       1477 non-null   float64
 9   Bedrooms                        1477 non-null   float64
 10  Beds                            1477 non-null   float64
 11  Bed Type                        1477 non-null   float64
 12  Guests Included                 14

En el siguiente bucle imputo los datos de train siguiendo el mismo procedimiento:

In [12]:
# Para cada columna:

for variable in list(X_train.columns):
    
    # Si, y sólo si la columna tiene valores faltantes:
    
    if sum(X_train[variable].isna()) != 0:
        print(variable)
        
        # Predigo los valores faltantes empleando un random forest:
        
        lista2 = list(X_train.columns)
        lista2.remove(variable)
        
        Predicciones = Imputar_RFR(
        X_train[variable][X_train[variable].isna()==False],
        X_train[lista2][X_train[variable].isna()==False],
        X_train[lista2][X_train[variable].isna()])
    
        # Sustituyo los valores faltantes por las predicciones:
        
        X_train.loc[X_train[variable].isna(),variable] = Predicciones

Host Response Rate
Host Listings Count
Host Total Listings Count
Bathrooms
Bedrooms
Beds
Review Scores Rating
Review Scores Accuracy
Review Scores Cleanliness
Review Scores Checkin
Review Scores Communication
Review Scores Location
Review Scores Value
Calculated host listings count
Reviews per Month


Compruebo que he eliminado los valores faltantes de las primeras 34 columnas (las siguientes columnas pertenecen a Amenities y Features y no tienen valores faltantes):

In [13]:
X_train.iloc[:,0:34].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13286 entries, 0 to 13285
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Host Response Time              13286 non-null  int64  
 1   Host Response Rate              13286 non-null  float64
 2   Host Listings Count             13286 non-null  float64
 3   Host Total Listings Count       13286 non-null  float64
 4   Host Verifications              13286 non-null  int64  
 5   Property Type                   13286 non-null  float64
 6   Room Type                       13286 non-null  int64  
 7   Accommodates                    13286 non-null  int64  
 8   Bathrooms                       13286 non-null  float64
 9   Bedrooms                        13286 non-null  float64
 10  Beds                            13286 non-null  float64
 11  Bed Type                        13286 non-null  float64
 12  Guests Included                 

Guardo los datasets con los datos imputados:

In [14]:
#X_train.to_csv('X_train_imputado.csv')

#X_test.to_csv('X_test_imputado.csv')